In [1]:
from discovery_utils.getters import hansard

In [9]:
import datetime

In [2]:
Hansard = hansard.HansardGetter()

In [3]:
debates_df = Hansard.get_debates_parquet()

2024-11-05 17:33:31,459 - discovery_utils.getters.hansard - INFO - Downloading debates parquet file: data/policy_scanning_data/enriched/HansardDebates.parquet


In [4]:
labelstore_df = Hansard.get_labelstore()

2024-11-05 17:34:43,253 - discovery_utils.getters.hansard - INFO - Attempting to download label store: data/policy_scanning_data/enriched/HansardDebates_LabelStore_keywords.csv


In [16]:
end_date = "2024-10-10"
# one week before the end date
start_date = "2024-10-03"

_debates_df = (
    debates_df[(debates_df["date"] >= start_date) & (debates_df["date"] <= end_date)]
    .merge(labelstore_df[["id", "mission_labels", "topic_labels"]], left_on="speech_id", right_on="id")
)

In [18]:
_debates_df.query("mission_labels == 'ASF'")

,speech_id,speakername,speaker_id,person_id,speech,date,year,major_heading,minor_heading,id,mission_labels,topic_labels
92,uk.org.publicwhip/debate/2024-10-07a.69.1,Bill Esterson,NA,uk.org.publicwhip/person/24905,"Unlike the shadow Secretary of State, I am ver...",2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.69.1,ASF,Hydrogen energy
93,uk.org.publicwhip/debate/2024-10-07a.69.2,Ed Miliband,NA,uk.org.publicwhip/person/11545,I thank my hon. Friend for his question; he ma...,2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.69.2,ASF,"CCUS,Hydrogen energy"
95,uk.org.publicwhip/debate/2024-10-07a.70.3,Ed Miliband,NA,uk.org.publicwhip/person/11545,May I begin by welcoming the hon. Lady to her ...,2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.70.3,ASF,CCUS
96,uk.org.publicwhip/debate/2024-10-07a.71.0,Barry Gardiner,NA,uk.org.publicwhip/person/10220,I had rather hoped that my right hon. Friend w...,2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.71.0,ASF,"CCUS,Hydrogen energy"
97,uk.org.publicwhip/debate/2024-10-07a.71.1,Ed Miliband,NA,uk.org.publicwhip/person/11545,I thank my hon. Friend for his question; he sp...,2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.71.1,ASF,Hydrogen energy
105,uk.org.publicwhip/debate/2024-10-07a.74.4,Dave Doogan,NA,uk.org.publicwhip/person/25796,The Secretary of State says he is absolutely c...,2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.74.4,ASF,CCUS
106,uk.org.publicwhip/debate/2024-10-07a.75.1,Anneliese Midgley,NA,uk.org.publicwhip/person/26318,I welcome my right hon. Friend’s announcement ...,2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.75.1,ASF,CCUS
107,uk.org.publicwhip/debate/2024-10-07a.75.3,Wera Hobhouse,NA,uk.org.publicwhip/person/25648,"Waste incinerators release, on average, a tonn...",2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.75.3,ASF,CCUS
109,uk.org.publicwhip/debate/2024-10-07a.76.1,Ed Miliband,NA,uk.org.publicwhip/person/11545,"I respect the right hon. Gentleman’s question,...",2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.76.1,ASF,CCUS
112,uk.org.publicwhip/debate/2024-10-07a.76.4,Adrian Ramsay,NA,uk.org.publicwhip/person/26514,"As the Secretary of State will be aware, the v...",2024-10-07,2024,"Carbon Capture, Usage and Storage",None,uk.org.publicwhip/debate/2024-10-07a.76.4,ASF,"CCUS,Hydrogen energy"
